# 🚀 Auto Manual Review
---
### The code is in the proper order.
### 👉 **Run each cell according to the instructions above each code cell.**

## 💻 Environment Setup

To run this notebook smoothly, we recommend the following environment:

### 🧠 IDE
- **[Visual Studio Code](https://code.visualstudio.com/)** (VS Code)
  A lightweight, powerful editor that supports Jupyter notebooks out of the box.

### 🧩 Required Extensions
- **Jupyter** extension (published by Microsoft)
  - Go to Extensions `(Ctrl+Shift+X)` → Search for `Jupyter` → Install.
- (Optional) **Python** extension (also by Microsoft) for syntax highlighting and Python support.

### 🧪 Python Environment
- Python version **3.9+** recommended.
- Use `venv`, `conda`, or your preferred environment manager to isolate dependencies.

### 🔁 Kernel Instructions
Once you open the notebook:
1. Click the top-right **kernel selector** (it may say “Python 3” or “Select Kernel”).
2. Choose the environment where you've installed your requirements.
3. If no environment appears, make sure it’s activated and Python is installed.

## ▶️ How to Use

This project is designed as a Jupyter Notebook, which runs Python code in cells. Here's how to interact with it:

### 🧾 Opening the Notebook
1. Launch **VS Code**.
2. Open the folder containing this project.
3. Open the `.ipynb` file (`auto_manual_review.ipynb` or similar).

### 🚀 Running Cells
- **Click** on a cell to select it.
- **Run a cell** by pressing:
  - `Shift + Enter` — runs the cell and moves to the next.
  - `Ctrl + Enter` — runs the cell but keeps the focus on it.
  - You can also use the **▶️ Run** button in the top bar.

### 📌 Important Notes
- **Run cells in order.** The notebook is designed to be executed from top to bottom.
- **Don't skip setup cells**, especially those that handle imports, functions, and cookie authentication.
- Output will appear directly below each cell when run.

# 📦 Cell 0: First-Time Setup 🚀
---
This cell ensures that all the required libraries are installed and ready to go for running the rest of the notebook.

✅ **What It Does**:
- Automatically checks for missing libraries:
  - `selenium`
  - `beautifulsoup4`
  - `pandas`
  - `requests`
  - `pyclip`
- Installs any missing ones using `pip`.

📌 **When to Use**:
- The very first time you run this notebook.
- Or if you’ve reinstalled Python or created a fresh environment.

🧰 **Outcome**:
All dependencies are ready — you're all set to move forward!

In [1]:
# 🚀 First-Time Setup: Check and Install Required Packages

import subprocess
import sys

# List of required packages
required_packages = ["selenium", "beautifulsoup4", "pandas", "requests", "pyclip", "ipywidgets", "ipyaggrid", "urllib3", "urllib", "re"]

def install_package(package):
    """Install package using pip."""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Try importing each package, install if not found
for package in required_packages:
    try:
        __import__(package.split('==')[0])
        print(f"✅ {package} is already installed.")
    except ImportError:
        print(f"📦 {package} not found. Installing...")
        install_package(package)
        print(f"✅ {package} installed successfully!")

print("\n🎉 Environment is ready!")

✅ selenium is already installed.
📦 beautifulsoup4 not found. Installing...
✅ beautifulsoup4 installed successfully!
✅ pandas is already installed.
✅ requests is already installed.
✅ pyclip is already installed.
✅ ipywidgets is already installed.
✅ ipyaggrid is already installed.
✅ urllib3 is already installed.
✅ urllib is already installed.
✅ re is already installed.

🎉 Environment is ready!


# 🛠️ Cell 1: Imports
---
Imports all necessary Python libraries for the script to function.

✅ **What It Includes**:
- Automation (`selenium`)
- Web scraping (`BeautifulSoup`)
- Data handling (`pandas`)
- API requests (`requests`)
- Clipboard access (`pyclip`)
- File and JSON operations

📌 **When to Use**:
Run this **once** every time you open the notebook.

In [2]:
from selenium import webdriver
from ipyaggrid import Grid
from selenium.webdriver.chrome.options import Options
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from urllib3.util.retry import Retry
from urllib.parse import quote_plus
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
import pyclip
import os
import json
import time
import ipaddress
import re

# 🧠 Cell 2: Function Definitions
---
This cell defines all the core functions used in the notebook.

✅ **What It Does**:
- Sets up reusable code blocks to:
  - Load and save cookies
  - Open Chrome and log in
  - Fetch reference data
  - Check registration and match percentages
  - Display results cleanly

📌 **When to Use**:
Run once at the beginning of each session.
Must be run **before any other code cells** that depend on these functions.

In [10]:
# ---- One-fetch-per-name helper (with cache) ----
try:
    import lxml  # noqa: F401
    _HAS_LXML = True
except Exception:
    _HAS_LXML = False

_LAST_REQUEST_TS = 0.0

def _extract_first_table_payload(soup_table):
    import re
    kv = parse_key_value_table(soup_table)

    # Total finishes
    raw_finishes = kv.get("Total finishes", "")
    total_finishes = None
    digits = "".join(ch for ch in raw_finishes if ch.isdigit())
    if digits:
        try:
            total_finishes = int(digits)
        except ValueError:
            total_finishes = None

    # Shared-with (DOM first, then KV)
    raw_shared_dom = extract_label_value(soup_table, "Same user or shared computer as/with")
    raw_shared_kv  = kv.get("Same user or shared computer as/with", "")
    raw_shared = (raw_shared_dom or raw_shared_kv or "").replace("\xa0", " ")
    raw_shared = re.sub(r"\s+", " ", raw_shared).strip()
    raw_shared = re.sub(r":\s*$", "", raw_shared)

    if raw_shared:
        parts = re.split(r",|\band\b", raw_shared, flags=re.IGNORECASE)
        same_user_with = [p.strip().strip('"\'' ) for p in parts if p.strip()]
    else:
        same_user_with = None

    return total_finishes, raw_finishes, raw_shared, same_user_with

def _candidate_usernames_from_page(soup):
    """
    Return (recent_names, top_names) from the page:
    - recent_names: unique names by most-recent appearance in the chronological table (best signal)
    - top_names: names sorted by finishes (desc) from the 'Finished on names' table
    """
    recent = []
    seen = set()
    # Chronological table is the 3rd <h1> section in your HTML
    h1s = soup.find_all("h1")
    if len(h1s) >= 3:
        chrono_tbl = h1s[2].find_next("table")
        if chrono_tbl:
            for tr in chrono_tbl.find_all("tr")[1:]:
                tds = tr.find_all("td")
                if len(tds) >= 2:
                    name = tds[1].get_text(" ", strip=True)
                    if name and name not in seen:
                        recent.append(name)
                        seen.add(name)

    top = []
    if len(h1s) >= 2:
        names_tbl = h1s[1].find_next("table")
        if names_tbl:
            for tr in names_tbl.find_all("tr")[1:]:
                tds = tr.find_all("td")
                if len(tds) == 2:
                    nm = tds[0].get_text(" ", strip=True)
                    fs = tds[1].get_text(" ", strip=True)
                    try:
                        fsi = int("".join(ch for ch in fs if ch.isdigit()) or "0")
                    except Exception:
                        fsi = 0
                    if nm:
                        top.append((nm, fsi))
            top.sort(key=lambda x: x[1], reverse=True)

    top_names = [n for n, _ in top]
    return recent, top_names

def _norm_username(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def _dedupe_preserve_order(items):
    seen = set()
    out = []
    for x in items:
        k = x.casefold()
        if k and k not in seen:
            seen.add(k)
            out.append(x)
    return out

def _parse_shared_with_cell(cell) -> list[str] | None:
    """
    Accepts a BeautifulSoup <td> that contains the 'shared with' value.
    Returns a list of usernames, or None if nothing meaningful.
    Handles anchors, <br>, commas, slashes, ampersands, and 'and'.
    """
    if not cell:
        return None

    names = []

    # 1) Explicit links (most reliable)
    for a in cell.select("a"):
        t = _norm_username(a.get_text(strip=True))
        if t:
            names.append(t)

    # 2) Text content (handles commas, '/', '&', 'and', <br>)
    #    Use get_text with a separator to keep <br> boundaries.
    text = cell.get_text(" ", strip=True)

    # If there were anchors, 'text' will include them too; splitting is still fine.
    # Split on common delimiters: commas, slashes, ampersands, "and" (word-boundary)
    parts = re.split(r"\s*,\s*|/|&|\band\b", text, flags=re.IGNORECASE)
    for p in parts:
        t = _norm_username(p)
        if t and t.lower() not in {"n/a", "na", "none", "-"}:
            names.append(t)

    names = _dedupe_preserve_order(names)
    return names or None

def extract_shared_with(soup: BeautifulSoup) -> list[str] | None:
    """
    Finds the table cell *after* the label cell that says something like:
    'Same user or shared computer as/with' (case-insensitive).
    Returns list of usernames or None.
    """
    # Find a <td> that looks like the label
    label_td = soup.find(
        lambda tag: tag.name == "td"
        and tag.get_text(strip=True)
        and re.search(r"same\s+user|shared\s+computer", tag.get_text(strip=True), re.I)
    )
    if not label_td:
        return None

    # Usually the value is the *next* <td> in the same row or the next sibling cell
    value_td = label_td.find_next("td")
    return _parse_shared_with_cell(value_td)

def mount_retries(session: requests.Session, total=5, backoff=1.5):
    retry = Retry(
        total=total, connect=total, read=total, status=total,
        backoff_factor=backoff,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=frozenset(["HEAD", "GET", "OPTIONS"]),
        respect_retry_after_header=True,
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=8, pool_maxsize=8)
    session.mount("https://", adapter)
    session.mount("http://", adapter)

def polite_get(session, url, min_interval=1.2, timeout=20):
    global _LAST_REQUEST_TS
    now = time.time()
    wait = (_LAST_REQUEST_TS + min_interval) - now
    if wait > 0:
        time.sleep(wait)
    resp = session.get(url, timeout=timeout)
    _LAST_REQUEST_TS = time.time()

    if resp.status_code == 429:
        ra = resp.headers.get("Retry-After")
        delay = int(ra) if ra and ra.isdigit() else 5
        time.sleep(delay)
        resp = session.get(url, timeout=timeout)
        _LAST_REQUEST_TS = time.time()

    resp.raise_for_status()
    return resp

def _norm_text(s: str) -> str:
    s = (s or "").replace("\xa0", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return re.sub(r":\s*$", "", s)

def _kv_get_fuzzy(kv: dict, label_substr: str) -> str:
    """Return the first KV value whose key contains label_substr (case-insensitive)."""
    target = label_substr.lower()
    for k, v in kv.items():
        if target in (k or "").lower():
            return v
    return ""

def is_na(s: str | None) -> bool:
    # normalize spaces (incl. NBSP) and case
    s = (s or "").replace("\xa0", " ")
    s = " ".join(s.split()).strip().lower()
    # common NA tokens (with or without punctuation)
    s_nospace = s.replace(" ", "")
    return (
        s in {"n/a", "na", "none", "-", "—", "n.a.", "n . a ."}
        or s_nospace in {"n/a", "na", "n/a/"}
    )

def parse_key_value_table(table):
    """Parse 2-col key/value tables, PLUS handle the 'value in next single-cell row' quirk."""
    import re
    def _clean(s: str) -> str:
        s = (s or "").replace("\xa0", " ")
        s = re.sub(r"\s+", " ", s).strip()
        s = re.sub(r":\s*$", "", s)
        return s

    out = {}
    last_key = None

    for tr in table.find_all("tr"):
        cells = tr.find_all(["td", "th"])
        # clean once per cell
        texts = []
        for c in cells:
            t = _clean(c.get_text(" ", strip=True))
            if t:
                texts.append(t)

        if len(texts) >= 2:
            key, val = texts[0], texts[1]
            if key:
                out[key] = val
                last_key = key
        elif len(texts) == 1:
            val = texts[0]
            if last_key is not None and (out.get(last_key) in (None, "", "N/A")):
                out[last_key] = val
        else:
            last_key = None

    return out

def extract_label_value(table, label_text: str) -> str:
    """
    Find the value cell for a row whose first cell contains `label_text`.
    Works even if the label cell has whitespace/newlines.
    """
    import re

    def _clean(s: str) -> str:
        s = (s or "").replace("\xa0", " ")
        s = re.sub(r"\s+", " ", s).strip()
        s = re.sub(r":\s*$", "", s)
        return s

    target = label_text.lower()

    for tr in table.find_all("tr"):
        cells = tr.find_all("td")
        if len(cells) >= 2:
            key = _clean(cells[0].get_text(" ", strip=True))
            if target in key.lower():
                return _clean(cells[1].get_text(" ", strip=True))
    return ""

    # 2) Very loose fallback: search any <td> whose text contains all key tokens
    tokens = [t for t in re.split(r"\s+|/|,", target) if t and t not in {":", "-"}]
    for td in table.find_all("td"):
        txt = _clean(td.get_text(" ", strip=True)).lower()
        if txt and all(tok in txt for tok in tokens[:3]):  # first few tokens should be enough
            # try next sibling value
            sib = td.find_next_sibling("td")
            if sib:
                val = _clean(sib.get_text(" ", strip=True))
                if val:
                    return val
    return ""

def save_cookies_to_file(cookies, filename="cookies.json"):
    with open(filename, "w") as f:
        json.dump(cookies, f)

def load_cookies_from_file(filename="cookies.json"):
    if os.path.exists(filename):
        with open(filename, "r") as f:
            return json.load(f)
    return None

def is_session_valid(session):
    try:
        resp = polite_get(session, "https://kog.tw/player_edit.php?player=", min_interval=1.2)
        return resp.status_code == 200 and "inputEmail" in resp.text
    except Exception:
        return False

def acquire_cookies():
    """Load saved cookies or log in via browser automatically."""
    # Try loading saved cookies
    cookies = load_cookies_from_file()
    if cookies:
        print("⌛ Loaded cookies from file. Verifying...")
        session = requests.Session()
        session.cookies.update(cookies)
        if is_session_valid(session):
            print("🍪 Cookies are still valid!")
            return cookies
        print("❌ Cookies expired or invalid. Need to log in again.")

    # Open Chrome for login
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://kog.tw")
    print("🌐 Browser opened. Please log in...")

    # Poll until login detected
    max_checks = 200  # ~10 minutes at 3s interval
    checks = 0
    cookies = None

    try:
        while checks < max_checks:
            time.sleep(3)  # check every 3 seconds
            checks += 1

            selenium_cookies = driver.get_cookies()
            session_cookies = {
                c['name']: c['value']
                for c in selenium_cookies
                if c['name'] in ('PHPSESSID', 'cf_clearance')
            }

            if session_cookies:
                session = requests.Session()
                session.headers.update({"User-Agent": "KoGTool/1.0"})
                session.cookies.update(session_cookies)
                mount_retries(session)

                if is_session_valid(session):
                    cookies = session_cookies
                    save_cookies_to_file(cookies)
                    print("✅ Login detected, cookies saved.")
                    break
        else:
            print("❌ Timed out waiting for login.")
            return None
    finally:
        # Always close the browser, success or timeout
        driver.quit()

    return cookies

def manual_cookie_fallback():
    print("Alternative method:")
    print("1. Visit https://kog.tw in Chrome")
    print("2. Open DevTools (F12 → Network tab)")
    print("3. Refresh and copy a request's 'Cookie' header")
    cookie_header = input("Paste cookie header here: ")
    return dict(pair.split("=", 1) for pair in cookie_header.split("; "))

def scrape_player_data(session, ref_number, username: str | None = None):
    url = f"https://kog.tw/player_migration.php?ref={quote_plus(str(ref_number))}"
    if username:  # only if not None/empty
        url += f"&username={quote_plus(username)}"

    resp = polite_get(session, url)
    html = resp.text
    parser = "lxml" if _HAS_LXML else "html.parser"
    soup = BeautifulSoup(html, parser)
    headers = soup.find_all("h1")

    if len(headers) < 1:
        print("❗ ERROR: No <h1> elements on the page.")
        return {"first_table": {}, "second_table": []}

    # ---- First table: anchor to "Hard facts" if present, else first <h1> ----
    first_table_data = {}
    hardfacts_h1 = soup.find(lambda tag: tag.name == "h1" and "hard facts" in tag.get_text(strip=True).lower())
    first_table = (hardfacts_h1.find_next("table") if hardfacts_h1 else headers[0].find_next("table"))

    if first_table:
        # Initial read (no username)
        total_finishes, raw_finishes, raw_shared, same_user_with = _extract_first_table_payload(first_table)

        # If missing/NA, build a prioritized candidate list and probe (bounded)
        need_probe = (not raw_shared) or is_na(raw_shared)
        if need_probe:
            recent_names, top_names = _candidate_usernames_from_page(soup)

            # Priority: explicit username (if given) → recent names → top names
            candidates = []
            if username:
                candidates.append(username)
            candidates.extend(recent_names)
            candidates.extend(top_names)

            # Dedup while preserving order
            seen = set()
            uniq_candidates = []
            for nm in candidates:
                nm = (nm or "").strip()
                if nm and nm.lower() not in seen:
                    uniq_candidates.append(nm)
                    seen.add(nm.lower())

            # Try at most N candidates, stop on first non-NA
            MAX_TRIES = 12
            for nm in uniq_candidates[:MAX_TRIES]:
                url2 = f"https://kog.tw/player_migration.php?ref={quote_plus(ref_number)}&username={quote_plus(nm)}"
                resp2 = polite_get(session, url2)
                soup2 = BeautifulSoup(resp2.text, "lxml" if _HAS_LXML else "html.parser")
                hardfacts_h1_b = soup2.find(lambda tag: tag.name == "h1" and "hard facts" in tag.get_text(strip=True).lower())
                first_table_b = (hardfacts_h1_b.find_next("table") if hardfacts_h1_b else soup2.find_all("h1")[0].find_next("table"))
                if not first_table_b:
                    continue

                tf, rf, rs, su = _extract_first_table_payload(first_table_b)
                if rs and not is_na(rs):
                    # lock in from the successful probe
                    total_finishes, raw_finishes, raw_shared, same_user_with = tf, rf, rs, su
                    break

        first_table_data = {
            "total_finishes": total_finishes,
            "same_user_or_shared_with": same_user_with,
            "raw": {
                "total_finishes": raw_finishes,
                "same_user_or_shared_with": raw_shared,
            },
        }
    else:
        print("❗ ERROR: No table found after the Hard facts <h1>.")

    # ---- Second table: anchored to the second <h1> ----
    second_table_rows = []
    if len(headers) >= 2:
        second_table = headers[1].find_next("table")
        if second_table:
            for row in second_table.find_all("tr")[1:]:  # skip header row if present
                tds = row.find_all("td")
                if len(tds) == 2:
                    second_table_rows.append({
                        "name": tds[0].get_text(strip=True),
                        "finishes": tds[1].get_text(strip=True),
                    })
        else:
            print("❗ ERROR: No table found after the second <h1>.")
    else:
        print("❗ ERROR: Not enough <h1> elements to locate the second table.")

    shared_with_list = extract_shared_with(soup)
    first_table["same_user_or_shared_with"] = shared_with_list

    return {
        "first_table": first_table_data,
        "second_table": second_table_rows,
    }

_player_soup_cache = {}

def _get_player_soup(session, player_name, min_interval=0.35):
    """Fetch and cache the player's edit page once per run."""
    from urllib.parse import quote_plus
    url = f"https://kog.tw/player_edit.php?player={quote_plus(player_name)}"

    soup = _player_soup_cache.get(url)
    if soup is not None:
        return soup

    resp = polite_get(session, url, min_interval=min_interval)
    html = resp.text
    parser = "lxml" if _HAS_LXML else "html.parser"
    soup = BeautifulSoup(html, parser)
    _player_soup_cache[url] = soup
    return soup

def check_status(session, player_name):
    try:
        soup = _get_player_soup(session, player_name)
        migration_label = soup.find("label", string="Migration Status")
        if migration_label:
            status_div = migration_label.find_next("div")
            migration_status = (status_div.get_text(strip=True) or "").strip('"')
            if migration_status == "Banned":
                return "🛑 BANNED"
            return ""  # migrated but not banned (adjust if you want to expose other states)
        else:
            return "⚠️ NOT MIGRATED"
    except Exception as e:
        print(f"Error checking {player_name}: {str(e)}")
        return "ERROR"

def check_player_info(session, player_name):
    try:
        soup = _get_player_soup(session, player_name)
        email_input = soup.find("input", {"name": "inputEmail"})
        if not email_input:
            return "❌ UNREGISTERED (NO EMAIL FIELD)"
        email = (email_input.get("value") or "").strip()
        return "✅ REGISTERED" if email else "❌ UNREGISTERED"
    except Exception as e:
        print(f"Error checking {player_name}: {str(e)}")
        return "ERROR"

def polite_post(session, url, json=None, headers=None, min_interval=1.2, timeout=20):
    # reuse the same global rate limiter
    global _LAST_REQUEST_TS
    now = time.time()
    wait = (_LAST_REQUEST_TS + min_interval) - now
    if wait > 0:
        time.sleep(wait)
    resp = session.post(url, json=json, headers=headers or {}, timeout=timeout)
    _LAST_REQUEST_TS = time.time()
    if resp.status_code == 429:
        ra = resp.headers.get("Retry-After")
        delay = int(ra) if ra and ra.isdigit() else 5
        time.sleep(delay)
        resp = session.post(url, json=json, headers=headers or {}, timeout=timeout)
        _LAST_REQUEST_TS = time.time()
    resp.raise_for_status()
    return resp

def check_player_ip(session, player_name, ip_address):
    """Check a player using IP address via kog.tw API."""
    try:
        url = "https://kog.tw/api.php?automated=1"
        payload = {
            "type": "user/admin/check_player",
            "data": {"playername": player_name, "playerip": ip_address},
        }
        headers = {"Content-Type": "application/json"}
        resp = polite_post(session, url, json=payload, headers=headers)
        data = resp.json()

        # Be defensive: extract something meaningful (adjust keys to match actual API)
        # Try common shapes first:
        #   {"ok": true, "match_percentage": 87}
        #   {"result": {"percentage": 87}}
        #   {"data": {"match": {"percent": 87}}}
        pct = (
            data.get("match_percentage")
            or (data.get("result") or {}).get("percentage")
            or ((data.get("data") or {}).get("match") or {}).get("percent")
        )

        # Normalize to int if it's numeric in string form
        if isinstance(pct, str) and pct.strip().isdigit():
            pct = int(pct.strip())

        return pct  # could be int, float, or None

    except Exception as e:
        print(f"❗ Error checking IP for {player_name}: {str(e)}")
        return None

def check_all_players(session, player_data, ip_address):
    """Check all players for registration status and IP match percentage."""
    results = []

    for player in player_data:
        name = player['name']
        finishes = player['finishes']

        # Check if player is registered
        status = check_player_info(session, name)

        # Default percentage = None
        percentage = None

        # If registered, check IP percentage
        if status.startswith("✅"):
            percentage = check_player_ip(session, name, ip_address)

        results.append({
            'name': name,
            'status': status,
            'finishes': finishes,
            'match_percentage': percentage
        })

    return results

def generate_output(review_name, player_data):
    norm_review = _norm_username(review_name)

    # remove the review name from the list (robust compare)
    filtered = [p for p in player_data if _norm_username(p.get('name')) != norm_review]

    output = (
        f"## Manual review is for: `{review_name}`\n"
        "Have you registered or completed a map with one or more of the following names?\n\n"
    )
    if filtered:
        output += "\n".join(f"- `{p['name']}`" for p in filtered)
    else:
        output += "_(no other names found)_"

    output += (
        "\n\n### Please elaborate your case:\n"
        "- If you already registered one of these names, why are you trying to register a new name?\n"
        "- If you just finished with one of these names, please do not finish maps for other names besides the one associated with your account.\n"
        "- If you did not register or finish maps for any of these names, please confirm that you did not register or finish any maps for these names.\n\n"
        "*While you are waiting for us, make sure to familiarize yourself with our [#kog-rulebook](https://discord.com/channels/342003344476471296/978628693389885490)*"
    )
    return output

def display_results_table(results):
    """Display table nicely in Jupyter"""
    if not results:
        print("❌ No results to display")
        return

    pd.set_option('display.max_rows', None)
    df = pd.DataFrame(results)

    col_defs = [{"field": c} for c in df.columns]

    grid = Grid(
        grid_data=df,
        grid_options={"columnDefs": col_defs},   # ✅ not empty
        quick_filter=True,
        width="50%",        # or "90%", 900, …
        height=400,          # pixels
        theme="ag-theme-balham-dark",
    )
    return grid

def display_full_results(results):
    """Display the full results as a DataFrame."""
    df = pd.DataFrame(results)

    col_defs = [{"field": c} for c in df.columns]

    grid = Grid(
        grid_data=df,
        grid_options={"columnDefs": col_defs},   # ✅ not empty
        quick_filter=True,
        width="50%",        # or "90%", 900, …
        height=400,          # pixels
        theme="ag-theme-balham-dark",
    )
    return grid

# 🍪 Cell 3: Grab Cookies
---
Handles user login and manages session cookies.

🔐 **How It Works**:
1. Checks if `cookies.json` already exists and is still valid.
2. If not, opens a Chrome window and prompts you to login at [kog.tw](https://kog.tw).
3. After logging in, you’ll type `done` to save cookies.

💾 **Outcome**:
Creates or updates `cookies.json` for future automated requests.

📌 **Important**:
This is only needed **if no cookies exist** or **your session expires**.

In [12]:
cookies = acquire_cookies()
if not cookies:
    cookies = manual_cookie_fallback()

⌛ Loaded cookies from file. Verifying...
🍪 Cookies are still valid!


# 🔎 Cell 4: Locate Review with Reference Number and Check Usernames
---
Looks up review data using a **reference number** and checks linked usernames.

🧠 **What It Does**:
- Finds the account linked to the reference number.
- Extracts all alternate usernames.
- Automatically skips the first name (it’s the same as the original account).
- Checks each one for **registration status**.

📌 **When to Use**:
After your cookies are working and you're ready to start a lookup.

In [20]:
try:
    from ipyaggrid import Grid
    _HAS_GRID = True
except Exception:
    _HAS_GRID = False

session = requests.Session()
session.cookies.update(cookies)

ref_number = input("Enter the reference number: ")
data = scrape_player_data(session, ref_number)   # if you have a review name, pass username=review_name

first_table = data.get("first_table", {}) or {}
second_table = data.get("second_table", []) or []

# ---- Build summary data ----
total_finishes = first_table.get("total_finishes")
shared_with = first_table.get("same_user_or_shared_with")  # list[str] or None
shared_with_str = (
    "N/A" if not shared_with else ", ".join(shared_with)
    if isinstance(shared_with, list) else (shared_with or "N/A")
)

summary_df = pd.DataFrame([{
    "Total finishes": total_finishes if total_finishes is not None else "Unknown",
    "Same user/shared with": shared_with_str,
}])

# ---- Display summary as a table ----
if _HAS_GRID:
    summary_grid = Grid(
        grid_data=summary_df,
        grid_options={"columnDefs": [{"field": c} for c in summary_df.columns]},
        theme="ag-theme-balham-dark",
        width="50%",
        height=100,
    )
    display(summary_grid)
else:
    display(summary_df)

# ---- Player rows (second table) ----
if not second_table:
    print("❌ No player data found in the second table.")
else:
    print("✅ Player data found!")

# Use the first row of the second table (if present) as a candidate review name
review_name = second_table[0]["name"] if second_table else None

print(f"⌛ [{ref_number}] Checking {len(second_table)} players...")
results = []

for player in second_table:
    status = check_player_info(session, player["name"])
    migration_status = check_status(session, player["name"])
    results.append({
        "name": player["name"],
        "status": status,
        "migration_status": migration_status,
        "finishes": player["finishes"],
    })

registered_name = [r for r in results if r["status"] == "✅ REGISTERED"]

print("✅ Check complete!")

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='100px', menu={'buttons': […

✅ Player data found!
⌛ [ref5189144] Checking 4 players...
✅ Check complete!


# 📝 Cell 5: Generate Copy/Paste Text
---
Generates a formatted message with the results from **Cell 4**, ready for pasting into chat.

🧾 **What It Includes**:
- Each checked username
- Their registration status
- Formatted neatly for readability

📋 **Bonus**:
Automatically copies the message to your clipboard for fast sharing.

📌 **When to Use**:
Right after checking usernames. Makes reporting results super quick!

In [21]:
first_table = data.get("first_table", {}) or {}
player_rows = data.get("second_table", []) or []   # <- this used to be `player_data`

# Pick a review name (first player), or fallback
review_name = player_rows[0]["name"] if player_rows else "(no players)"

# --- build the manual-review text ---
full_output = generate_output(review_name, player_rows)
display(Markdown(full_output))

try:
    pyclip.copy(full_output)
    print("📋 Copied to clipboard.\n")
except Exception:
    print("❌ Could not copy to clipboard (pyclip error).")

# --- compute results if you haven't already ---
results = []
for player in player_rows:
    status = check_player_info(session, player["name"])
    migration_status = check_status(session, player["name"])
    results.append({
        "name": player["name"],
        "status": status,
        "migration_status": migration_status,
        "finishes": player["finishes"],
    })

# --- show results ---
display(Markdown("---\n### Results:"))
grid = display_results_table(results)   # returns a Grid object
if grid is not None:
    display(grid)

## Manual review is for: `limon kurdu`
Have you registered or completed a map with one or more of the following names?

- `Joppy`
- `Dywka jadix`
- `Dywka afk 1 min`

### Please elaborate your case:
- If you already registered one of these names, why are you trying to register a new name?
- If you just finished with one of these names, please do not finish maps for other names besides the one associated with your account.
- If you did not register or finish maps for any of these names, please confirm that you did not register or finish any maps for these names.

*While you are waiting for us, make sure to familiarize yourself with our [#kog-rulebook](https://discord.com/channels/342003344476471296/978628693389885490)*

📋 Copied to clipboard.



---
### Results:

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='400px', menu={'buttons': […

# 🗂️ Cell 6: Collect Registered Usernames Only
---
Pulls out just the usernames that came back as ✅ REGISTERED in Cell Y and stores them in a tidy list for anything you need next.

**🧾 What It Does**:
- Scans the full results list
- Filters for entries whose status is ✅ REGISTERED
- Builds a clean, duplicate-free list called registered_names

**⚙️ Under the Hood**: Uses a one-liner list-comprehension (or a loop, if you prefer) so you always get the freshest set of confirmed names.

**📋 Bonus**: Returns the list in the same order the users were checked—handy if you need to keep things sequential.

**📌 When to Run It**: Immediately after your status-checking cell finishes. Run this, then hand the list off to downstream tasks like generating the manual-review template or syncing to your database.

In [22]:
registered_output = generate_output(review_name, registered_name)
display(Markdown(registered_output))

try:
    pyclip.copy(registered_output)
    print("📋 Copied to clipboard.")
except:
    print("❌ Could not copy to clipboard (pyclip error).")

## Manual review is for: `limon kurdu`
Have you registered or completed a map with one or more of the following names?

- `Joppy`
- `Dywka jadix`

### Please elaborate your case:
- If you already registered one of these names, why are you trying to register a new name?
- If you just finished with one of these names, please do not finish maps for other names besides the one associated with your account.
- If you did not register or finish maps for any of these names, please confirm that you did not register or finish any maps for these names.

*While you are waiting for us, make sure to familiarize yourself with our [#kog-rulebook](https://discord.com/channels/342003344476471296/978628693389885490)*

📋 Copied to clipboard.


# 📊 Cell 7: Display Status Report *(Optional)*
---

This cell takes the results from **Cell 5** — which contains the registration status of each username — and displays them in a neatly formatted table using **Pandas**.

✅ **Purpose**:
To quickly scan and verify which usernames are registered or unregistered, along with their associated map finishes.

🧠 **What It Does**:
- Converts the list of results (name, status, finishes) into a `DataFrame`.
- Outputs the table with improved formatting in Jupyter Notebook (not just plain text).

📌 **When to Use**:
- After you’ve run **Cell 5** and want a clearer visual overview.
- Optional step for better readability — does not affect any later cells.

🖥️ **Output Example**:
| name       | status             | finishes |
|------------|--------------------|----------|
| `Player1`  | ✅ REGISTERED       | 12       |
| `Player2`  | ❌ UNREGISTERED     | 0        |

In [23]:
display_results_table(results)

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='400px', menu={'buttons': […

# 🗂️ Cell 8: Generate Verification Prompt *(Optional)*
---
Creates a detailed **follow-up message** to send to a user who has map finishes on multiple registered usernames.

🧠 **What It Does**:
- Lists all **registered usernames** from the current result.
- Adds a polite prompt asking the user for clarification.
- Formats the message in Markdown with backticks and emphasis.
- Attempts to **copy** the entire message to your clipboard.

📋 **Message Example**:
```markdown
### You have map finishes on the following registered usernames: `User1`, `User2`, `User3`

*Are you aware that only one account is allowed per player?*

What can you tell me about these accounts?
```

💡 **Clipboard Support**:
If successful, it prints: `📋 Copied to clipboard.`
If there's an issue with `pyclip`, it prints an error instead.

📌 **When to Use**:
After verifying registrations — this cell is useful for **manual outreach** or moderation follow-ups.

In [29]:
verification = f"### You have map finishes on the following registered usernames: "
verification += ", ".join(f"`{p['name']}`" for p in results if p['status'] == "✅ REGISTERED")

verification += f"\n*Are you aware that only one account is allowed per player?*"

verification += (f"\n\nWhat can you tell me about these accounts?")

print(verification)

try:
    pyclip.copy(verification)
    print("\n📋 Copied to clipboard.")
except:
    print("\n❌ Could not copy to clipboard (pyclip error).")

### You have map finishes on the following registered usernames: `бесквит`
*Are you aware that only one account is allowed per player?*

What can you tell me about these accounts?

📋 Copied to clipboard.


# 🧮 Cell 9: Display Full Report *(Optional)*
---
Expands the previous report by checking **match percentages** for all registered usernames.

📊 **What It Adds**:
- Percentage match for each name (via the API).
- Appends this data to the same table from **Cell 6**.

🎯 **Useful For**:
- Deeper analysis
- Flagging suspicious overlaps

📌 **When to Use**:
After you’ve checked usernames and want full details in one place.

In [12]:
ip_input = input("Enter IP address to check: ").strip()

try:
    if ip_input:
        # validate IP (accepts v4 or v6; use ipaddress.IPv4Address if you want v4-only)
        try:
            ip_str = str(ipaddress.ip_address(ip_input))
        except ValueError:
            print("❌ Invalid IP address.")
        else:
            print("Checking IP address against usernames...")
            full_results = check_all_players(session, player_rows, ip_str)  # <- was player_data
            grid = display_full_results(full_results)
            if grid is not None:
                display(grid)
    else:
        print("❌ No IP address provided.")
except Exception as e:
    print("Error:", e)

Checking IP address against usernames...


Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='400px', menu={'buttons': […